### 네이버 함께 많이 찾는 주제어 수집
- 정적(static) 웹페이지 데이터 수집 
- BeautifulSoup을 이용하여 HTML 문자열 데이터 parsing

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

#### 1. 웹페이지 분석 : URL

In [2]:
url = "https://search.naver.com/search.naver?query=삼성전자"

#### 2. request(URL) > response : str(html)

In [3]:
response = requests.get(url)
response

<Response [200]>

In [7]:
response.text[:300]

'<!doctype html> <html lang="ko"><head> <meta charset="utf-8"> <meta name="referrer" content="always">  <meta name="format-detection" content="telephone=no,address=no,email=no"> <meta property="og:title" content="삼성전자 : 네이버 검색"/> <meta property="og:image" content="https://ssl.pstatic.net/sstatic/sear'

#### 3. str(html) > bs object

In [8]:
dom = BeautifulSoup(response.text, 'html.parser')
type(dom)

bs4.BeautifulSoup

#### 4. bs object > .select(css-selector), .select_one(css-selector) > str(text)

In [13]:
# select > list(Tag1, Tag2, ...), select_one > Tag
#fdr-53bf315101004d2ea860c7d9458c5c73 > div > div > div.bAUlEC095ho5bG7U0km1.fds-grid-layout-keyword.fds-refine-query-grid > div:nth-child(1) > div:nth-child(1) > a
# id > div > div > div이면서.class.class들을 포함 > 1번째중에 div면 선택
# 실제로는 길게 쓰지 않아도 됨
css_selector = ".fds-refine-query-grid span" # 하위 span을 모두 찾아준다.
elements = dom.select(css_selector)
len(elements), elements

(6,
 [<span class="fds-keyword-text EC6qOsUFAOwAI0C5oRtw">삼성전자주가</span>,
  <span class="fds-keyword-text EC6qOsUFAOwAI0C5oRtw">오늘 삼성전자 주가</span>,
  <span class="fds-keyword-text EC6qOsUFAOwAI0C5oRtw">삼성전자주식가격</span>,
  <span class="fds-keyword-text EC6qOsUFAOwAI0C5oRtw">삼성전자 주식</span>,
  <span class="fds-keyword-text EC6qOsUFAOwAI0C5oRtw">삼성전자 주가 전망</span>,
  <span class="fds-keyword-text EC6qOsUFAOwAI0C5oRtw">삼성전자주식전망</span>])

#### 5. str(text) > DataFrame

In [17]:
elements[0].text

'삼성전자주가'

In [18]:
[element.text for element in elements]

['삼성전자주가', '오늘 삼성전자 주가', '삼성전자주식가격', '삼성전자 주식', '삼성전자 주가 전망', '삼성전자주식전망']

In [24]:
# 6. function

def find_relational_text(keyword):
    url = f"https://search.naver.com/search.naver?query={keyword}"
    
    response = requests.get(url)
    dom = BeautifulSoup(response.text, 'html.parser')
    
    css_selector = ".fds-refine-query-grid span" # 하위 span을 모두 찾아준다.
    elements = dom.select(css_selector)
    
    return [element.text for element in elements]

In [25]:
find_relational_text("엔비디아")

['엔비디아주가',
 '엔비디아 실적발표 시간',
 '엔비디아 실적발표',
 '엔비디아 주가 전망',
 '엔비디아 실적발표 한국시간',
 '엔비디아 실적']

In [28]:
find_relational_text("SK")

['sk엔카 중고차',
 'SK하이닉스 주가',
 'sk이노베이션 매수청구권',
 'SK텔레콤 주가',
 'sk브로드밴드 인터넷',
 'sk엔카 직영몰']

In [ ]:
# daum exchange : headers (forbidden 되었을 때 header를 추가)
# naver restapi : application 등록 > key 입력 (찝찝하다면 api를 사용)
# zigbang : request > response : 여러번 (데이터를 한번에 가져올 수 없었음, 복잡한 형태)
# static page crawling : html > BeautifulSoup > css-selector (naver relational text)